In [3]:
from ax.service.ax_client import AxClient, ObjectiveProperties
from ax.utils.measurement.synthetic_functions import hartmann6
from ax.utils.notebook.plotting import init_notebook_plotting, render

import numpy as np

import requests

Mock 'driver'/ ax service client. Call the 'launch new experiment' endpoint, then query the /update campaign endpoint with new data 

In [2]:
def evaluate(parameterization):
    x = np.array([parameterization.get(f"x{i+1}") for i in range(6)])
    return {"hartmann6": (hartmann6(x), 0.0), "l2norm":(np.sqrt((x**2).sum()), 0.0)}

In [36]:
# 1. set new experiment

response = requests.post('http://127.0.0.1:5000/new_experiment')

In [37]:
response.content

b'Initialized new experiment'

In [47]:
trial_data = requests.get('http://127.0.0.1:5000/get_next_trial')

In [48]:
trial_data

<Response [200]>

In [49]:
trial_data.json()

{'parameterization': {'x1': 0.5437300326302648,
  'x2': 0.08437505271285772,
  'x3': 0.5857523065060377,
  'x4': 0.3651215313002467,
  'x5': 0.7211774857714772,
  'x6': 0.21486885007470846},
 'trial_index': 1}

In [50]:
parameterization = trial_data.json()['parameterization']
trial_index = trial_data.json()['trial_index']

In [51]:
parameterization

{'x1': 0.5437300326302648,
 'x2': 0.08437505271285772,
 'x3': 0.5857523065060377,
 'x4': 0.3651215313002467,
 'x5': 0.7211774857714772,
 'x6': 0.21486885007470846}

In [52]:
results = evaluate(parameterization)

In [53]:
results

{'hartmann6': (-0.061932165241241476, 0.0), 'l2norm': (1.159933870449813, 0.0)}

In [54]:
result_data = {'trial_index':trial_index, 'results':results}

In [55]:
requests.get('http://127.0.0.1:5000/complete_trial', json = result_data)

<Response [200]>